# EDA and first look / analysis

In [ ]:
import pandas as pd
import numpy as np
import os
import sys
import argparse
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# read data 

df=pd.read_csv("../../data/clean/beer_profile_and_ratings.csv")

In [ ]:
df

In [ ]:
# Vista general
print("Shape:", df.shape)
df.head()

In [ ]:
# Tipos de datos
print(df.dtypes)

In [ ]:
# Valores nulos
print(df.isna().sum())


In [ ]:
# Estadísticas generales
df.describe(include='all').T

In [ ]:
df.columns

###  Tabla de variables del dataset `beer_profile_and_ratings.csv`

| Variable              | Tipo       | Descripción                                                                 |
|-----------------------|------------|-----------------------------------------------------------------------------|
| `name`                | Categórica | Nombre corto de la cerveza                                                 |
| `style`               | Categórica | Estilo de cerveza (e.g., IPA, Lager, Stout)                                |
| `brewery`             | Categórica | Nombre de la cervecería                                                    |
| `beer name (full)`    | Texto      | Nombre completo o extendido de la cerveza                                  |
| `description`         | Texto      | Descripción sensorial de la cerveza                                        |
| `abv`                 | Numérica   | Alcohol by Volume (%), contenido alcohólico                                |
| `min ibu`             | Numérica   | Valor mínimo de International Bitterness Units si se conoce                |
| `max ibu`             | Numérica   | Valor máximo de IBU si se conoce                                           |
| `astringency`         | Numérica   | Sensación de sequedad en boca (0–5)                                        |
| `body`                | Numérica   | Cuerpo o densidad percibida (0–5)                                          |
| `alcohol`             | Numérica   | Percepción del alcohol (0–5)                                               |
| `bitter`              | Numérica   | Intensidad del sabor amargo (0–5)                                          |
| `sweet`               | Numérica   | Intensidad del sabor dulce (0–5)                                           |
| `sour`                | Numérica   | Intensidad del sabor ácido (0–5)                                           |
| `salty`               | Numérica   | Percepción salada (0–5)                                                    |
| `fruits`              | Numérica   | Intensidad de notas frutales (0–5)                                         |
| `hoppy`               | Numérica   | Intensidad de lúpulo (0–5)                                                 |
| `spices`              | Numérica   | Percepción de especias (0–5)                                               |
| `malty`               | Numérica   | Intensidad de maltosidad (0–5)                                             |
| `review_aroma`        | Numérica   | Calificación del aroma por usuarios (0–5)                                  |
| `review_appearance`   | Numérica   | Calificación de apariencia por usuarios (0–5)                              |
| `review_palate`       | Numérica   | Calificación del paladar por usuarios (0–5)                                |
| `review_taste`        | Numérica   | Calificación del sabor por usuarios (0–5)                                  |
| `review_overall`      | Numérica   | Calificación global por usuarios (0–5)                                     |
| `number_of_reviews`   | Entera     | Número total de reseñas registradas                                        |


In [ ]:
# Contar valores únicos en columnas categóricas clave
unique_names = df['name'].nunique()
unique_styles = df['style'].nunique()
unique_breweries = df['brewery'].nunique()

(unique_names, unique_styles, unique_breweries)

In [ ]:
# Renombrar las columnas del DataFrame a minúsculas y con guiones bajos
df.columns = (
    df.columns
    .str.strip()
    .str.lower()
    .str.replace(" ", "_")
    .str.replace("(", "")
    .str.replace(")", "")
)

# Verificar los nuevos nombres de columna
df.columns


In [ ]:
# Variables numéricas sensoriales y de reseña
numerical_cols = [
    'abv', 'min_ibu', 'max_ibu',
    'astringency', 'body', 'alcohol', 'bitter',
    'sweet', 'sour', 'salty', 'fruits', 'hoppy', 'spices', 'malty',
    'review_aroma', 'review_appearance', 'review_palate',
    'review_taste', 'review_overall', 'number_of_reviews'
]

# Histograma para variables seleccionadas
df[numerical_cols].hist(bins=20, figsize=(18, 14), color='skyblue', edgecolor='black')
plt.suptitle("Distribución de variables numéricas", fontsize=16)
plt.tight_layout()
plt.show()


In [ ]:
# Calcular matriz de correlación para las variables numéricas
corr_matrix = df[numerical_cols].corr()

# Visualizar la matriz con un mapa de calor
plt.figure(figsize=(16, 12))
sns.heatmap(corr_matrix, annot=True, fmt=".2f", cmap="coolwarm", center=0)
plt.title("Matriz de correlación entre variables sensoriales y de reseñas", fontsize=16)
plt.xticks(rotation=45, ha='right')
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()


---

### Selección de columnas relevantes 

Creamos un subconjunto df_features con variables numéricas relevantes para definir el perfil de sabor y evaluación de cada cerveza.

In [ ]:
# Selección de columnas sensoriales y de calificación
feature_cols = [
    'abv', 'astringency', 'body', 'alcohol', 'bitter',
    'sweet', 'sour', 'salty', 'fruits', 'hoppy', 'spices',
    'malty', 'review_aroma', 'review_appearance', 'review_palate',
    'review_taste', 'review_overall'
]

# Subconjunto con esas columnas
df_features = df[feature_cols]
df_features.head()


In [ ]:
# Revisar tipos y nulos antes
print(df_features.dtypes)
print(df_features.isna().sum())

# Convertir todo a numérico (cualquier error se convierte a NaN)
df_features_clean = df_features.apply(pd.to_numeric, errors='coerce')

# Revisar nulos después de la conversión
print(df_features_clean.isna().sum())

###  Escalar los datos sensoriales

In [ ]:
from sklearn.preprocessing import StandardScaler

# Instanciar el escalador
scaler = StandardScaler()

# Ajustar y transformar los datos
X_scaled = scaler.fit_transform(df_features_clean)

# Convertir a DataFrame para conservar los nombres
df_scaled = pd.DataFrame(X_scaled, columns=df_features_clean.columns)

# Ver primeras filas
df_scaled.head()


## KNN 

La métrica de distancia del coseno se utiliza en este modelo de recomendación porque permite comparar cervezas con base en la **forma general de su perfil sensorial**, sin verse afectada por la magnitud absoluta de cada atributo. A diferencia de la distancia euclidiana, que mide diferencias en valores exactos, la distancia del coseno se enfoca en el **ángulo entre los vectores de características**, lo que es útil cuando dos cervezas tienen proporciones similares entre atributos como `bitter`, `sweet` o `hoppy`, aunque sus valores absolutos sean distintos. Esto resulta especialmente adecuado para datos previamente escalados, y para espacios de alta dimensión como este, donde la métrica coseno tiende a ser más robusta y representativa para evaluar similitud relativa entre elementos.


In [ ]:
from sklearn.neighbors import NearestNeighbors

# Instanciar el modelo KNN con métrica coseno
knn_model = NearestNeighbors(n_neighbors=6, metric='cosine')  # incluye la cerveza base
knn_model.fit(df_scaled)

# Elegir un índice de referencia (puedes probar con otros)
index_ref = 0

# Encontrar vecinos más cercanos
distances, indices = knn_model.kneighbors(df_scaled.iloc[[index_ref]])

# Mostrar nombres y distancias
print("Cerveza base:", df.iloc[index_ref]['name'])
print("\nCervezas recomendadas:")

for i, idx in enumerate(indices[0][1:], 1):  # excluye la cerveza base (índice 0)
    print(f"{i}. {df.iloc[idx]['name']} (Distancia: {distances[0][i]:.4f})")


#### Distancia promedio de las recomendaciones

In [ ]:
def score_similitud(index_ref, df_scaled, model, top_k=5):
    distances, indices = model.kneighbors(df_scaled.iloc[[index_ref]])
    mean_distance = distances[0][1:top_k+1].mean()
    return mean_distance


In [ ]:
index_ref = df[df['name'] == "amber"].index[0]
score_similitud(index_ref, df_scaled, knn_model)


#### Evaluación masiva del recomendador
Calcula la distancia promedio coseno entre cada cerveza y sus k recomendaciones más cercanas. Esto te dice qué tan similares son, en promedio, las recomendaciones que entrega el modelo.

In [ ]:
import numpy as np

def score_similitud(index_ref, df_scaled, model, top_k=5):
    distances, indices = model.kneighbors(df_scaled.iloc[[index_ref]])
    return distances[0][1:top_k+1].mean()

def evaluar_recomendador(df_scaled, model, muestras=100, top_k=5, seed=42):
    np.random.seed(seed)
    indices = np.random.choice(len(df_scaled), size=muestras, replace=False)
    scores = [score_similitud(i, df_scaled, model, top_k=top_k) for i in indices]
    return {
        'distancias': scores,
        'promedio': np.mean(scores),
        'desviacion': np.std(scores)
    }


In [ ]:
resultados = evaluar_recomendador(df_scaled, knn_model, muestras=100, top_k=5)

print("Distancia promedio entre recomendaciones:", resultados['promedio'])
print("Desviación estándar:", resultados['desviacion'])


---

# Kmeans

In [ ]:
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

k_range = range(2, 11)
inertias = []

for k in k_range:
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    kmeans.fit(df_scaled)
    inertias.append(kmeans.inertia_)

plt.figure(figsize=(8, 5))
plt.plot(k_range, inertias, marker='o')
plt.xlabel("Número de clusters (k)")
plt.ylabel("Inercia")
plt.title("Método del codo para determinar k óptimo")
plt.grid(True)
plt.show()


In [ ]:
from sklearn.cluster import KMeans

# Entrenar modelo final
kmeans = KMeans(n_clusters=5, random_state=42, n_init=10)
df['cluster'] = kmeans.fit_predict(df_scaled)

# Calcular promedios sensoriales por cluster
df_clusters = pd.DataFrame(df_scaled, columns=feature_cols)
df_clusters['cluster'] = df['cluster']
cluster_means = df_clusters.groupby('cluster').mean()

# Graficar cada cluster como un perfil de barra
cluster_means.T.plot(kind='bar', figsize=(14, 6))
plt.title("Perfil sensorial promedio por cluster")
plt.xlabel("Atributo sensorial")
plt.ylabel("Valor estandarizado")
plt.xticks(rotation=45, ha='right')
plt.legend(title='Cluster')
plt.tight_layout()
plt.show()


In [ ]:
from sklearn.metrics import silhouette_score
score = silhouette_score(df_scaled, df['cluster'])
print("Silhouette Score:", score)


In [ ]:
df['cluster'].value_counts()
df.groupby('cluster')['style'].value_counts()


---

## Estrategia recomendador híbrido: Cluster Kmeans + KNN

In [ ]:
df_scaled_df = pd.DataFrame(df_scaled, columns=feature_cols, index=df.index)


In [ ]:
from sklearn.neighbors import NearestNeighbors

def recomendar_hibrido(nombre_cerveza, df, df_scaled_df, top_n=5):
    # Buscar cerveza base
    base = df[df['name'].str.contains(nombre_cerveza, case=False)]
    if base.empty:
        print("❌ No se encontró la cerveza.")
        return
    
    index_ref = base.index[0]
    nombre = base.loc[index_ref, 'name']
    cluster_id = base.loc[index_ref, 'cluster']
    
    print(f"🍺 Cerveza base: {nombre}")
    print(f"🧠 Cluster sensorial asignado: {cluster_id}\n")

    # Subconjunto del cluster
    cluster_df = df[df['cluster'] == cluster_id]
    cluster_scaled = df_scaled_df.loc[cluster_df.index]

    # Reentrenar KNN sobre ese cluster
    knn = NearestNeighbors(n_neighbors=top_n + 1, metric='cosine')
    knn.fit(cluster_scaled)

    # Encontrar posición relativa en cluster
    pos_in_cluster = list(cluster_df.index).index(index_ref)
    distances, indices = knn.kneighbors(cluster_scaled.iloc[[pos_in_cluster]])


    # Mostrar recomendaciones (excluyendo la base)
    for i, idx in enumerate(indices[0][1:], 1):
        cerveza_idx = cluster_df.index[idx]
        print(f"{i}. {df.loc[cerveza_idx, 'name']} ({df.loc[cerveza_idx, 'style']}) - Distancia: {distances[0][i]:.4f}")


In [ ]:
recomendar_hibrido("amber", df, df_scaled_df, top_n=5)


In [ ]:
# Lista de cervezas populares por nombre
ejemplos = ["stone ipa", "guinness", "sierra nevada", "duvel", "pumpkin", "chocolate stout"]

for nombre in ejemplos:
    print("="*60)
    recomendar_hibrido(nombre, df, df_scaled_df, top_n=5)


In [ ]:
df['name'].nunique()
